In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob
from imblearn.over_sampling import (RandomOverSampler, SMOTENC)
from sklearn.linear_model import (ElasticNet, Ridge, Lasso, LogisticRegression)
from sklearn.ensemble import (RandomForestRegressor, RandomForestClassifier)
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn.svm import SVR
from sklearn.model_selection import RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
import json
from category_encoders import *

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import *
from sklearn.naive_bayes import GaussianNB
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import balanced_accuracy_score
import seaborn as sns

In [3]:
def display_metrics(classifierName,Y_pred,Y_true):
    print("______________________________________________")
    print(("Classifier: "+classifierName))
    if np.array_equal(Y_pred,Y_true):
        print("Match")
        
    acc, precision, recall, f1score,bal_accuracy = classification_metrics(Y_pred,Y_true)
    mc_points = Y_pred==Y_true
    mc_p=np.size(mc_points) - np.count_nonzero(mc_points)
    print(("Accuracy: "+str(acc)))
    #print(("AUC: "+str(auc_)))
    print(("Precision: "+str(precision)))
    print(("Recall: "+str(recall)))
    print(("F1-score: "+str(f1score)))
    print(("Balanced Accuracy: "+str(bal_accuracy)))
    #print(("Misclassified Points: "+str(mc_p)))
    
    print("______________________________________________")
    print("")
    
    return acc, precision, recall, f1score, bal_accuracy

In [4]:
def classification_metrics(Y_pred,Y_true):
    cnf_matrix = metrics.confusion_matrix(Y_true, Y_pred)
    accuracy=metrics.accuracy_score(Y_true, Y_pred)
    bal_accuracy= metrics.balanced_accuracy_score(Y_true, Y_pred)
    precision=metrics.precision_score(Y_true, Y_pred,average='weighted')
    recall=metrics.recall_score(Y_true, Y_pred,average='weighted')
    fscore=metrics.f1_score(Y_true, Y_pred,average='weighted')
    #auc = metrics.roc_auc_score(Y_true, Y_pred,average='micro')
    return accuracy,precision,recall,fscore,bal_accuracy

In [17]:
def dataprep(df,airport):
    print(df.columns)
    #new_header = df.iloc[0]
    #df = df[1:].reset_index(drop = True)
    #df.columns = new_header
    #.rename(columns = df.iloc[0])
    y = df['Avg Gate Arrival Delay']
    df = df.iloc[:, :24] ## Take Columns A-X
    df['Avg Gate Arrival Delay'] = y
    df = df.rename(columns = {'WX\nCodes':'WXCodes', 'Sched.\nDepartures': 'SchedDepart', 'Sched.\nArrivals': 'SchedArriv'})
    ## Removing Nulls

    # It would make sense to remove null rows for Sced. Departures and Sched Arrivals as we Can't really replace them with anything.
    
    df = df[~df['SchedDepart'].isnull()]

    #1. There are common null values for METAR data. Only 24 hour of data provided for METAR. Remove these rows after midnight
    # here I am taking Vis column but any other METAR column shares same NaNs after midnight.
    df = df[~df['Vis'].isnull()]

    # Sky 1 NaN values 
    df = df[~df['Sky 1'].isnull()]

    # Dealing with NAN
    # Recall Wind Gust Finite values are excessive wind speed from Col F. Null values are steady state. Fillna with zeros as there are no excessive wind
    df['Wind Gust'] = df['Wind Gust'].fillna(0)

    # Similarly Peak Gust Speed's NaN can be treated as 0.
    df['Peak Gust Speed'] = df['Peak Gust Speed'].fillna(0)

    # Layer 1 is NaN if Sky 1 is CLR (No Clouds below 12000 ft). Let's fill na with max(Layer 1)
    df['Layer 1'].fillna(df['Layer 1'].max(), inplace = True)

    df['Ice 1 Hour'].fillna(0, inplace = True)
    df['Ice 3 Hour'].fillna(0, inplace = True)
    df['Ice 6 Hour'].fillna(0, inplace = True)

    # drop unrelevant columns
    df.drop(columns = ['Time', 'Layer 2', 'Layer 3', 'Sky 2', 'Sky 3', 'Feels Like', 'Ice 1 Hour', 'Ice 3 Hour', 'Ice 6 Hour'], inplace = True)

    # Deal with nulls in Sky 1
    # Deal with WxCodes
    df['WXCodes'].fillna('Regular', inplace = True)

    # Converting Objects to Floats
    for col in df.columns:
        if col != 'Sky 1' and col != 'WXCodes' and col != 'Peak Gust Direction':
            df[col] = df[col].astype(float)




    # Save response Variable seperately and remove from dataframe
    df['y'] = df['Avg Gate Arrival Delay'].div(10).apply(np.floor)
    y = df['y']
    #print(y)
    del df['Avg Gate Arrival Delay']
    del df['y']

    # Bucketting Wind Direction and Peak Gust WindDirection

    df.loc[(df['Peak Gust Direction'] >= 0) & (df['Peak Gust Direction'] < 11.25),'PGDirectionBkt' ]='N'
    df.loc[(df['Peak Gust Direction'] >= 11.25) & (df['Peak Gust Direction'] < 33.75),'PGDirectionBkt' ]='NNE'
    df.loc[(df['Peak Gust Direction'] >= 33.75) & (df['Peak Gust Direction'] < 56.25),'PGDirectionBkt' ]='NE'
    df.loc[(df['Peak Gust Direction'] >= 56.25) & (df['Peak Gust Direction'] < 78.75),'PGDirectionBkt' ]='ENE'
    df.loc[(df['Peak Gust Direction'] >= 78.75) & (df['Peak Gust Direction'] < 101.25),'PGDirectionBkt' ]='E'
    df.loc[(df['Peak Gust Direction'] >= 101.25) & (df['Peak Gust Direction'] < 121.75),'PGDirectionBkt' ]='ESE'
    df.loc[(df['Peak Gust Direction'] >= 123.75) & (df['Peak Gust Direction'] < 146.25),'PGDirectionBkt' ]='SE'
    df.loc[(df['Peak Gust Direction'] >= 146.25) & (df['Peak Gust Direction'] < 168.75),'PGDirectionBkt' ]='SSE'
    df.loc[(df['Peak Gust Direction'] >= 168.75) & (df['Peak Gust Direction'] < 191.25),'PGDirectionBkt' ]='S'
    df.loc[(df['Peak Gust Direction'] >= 191.25) & (df['Peak Gust Direction'] < 213.75),'PGDirectionBkt' ]='SSW'
    df.loc[(df['Peak Gust Direction'] >= 213.75) & (df['Peak Gust Direction'] < 236.25),'PGDirectionBkt' ]='SW'
    df.loc[(df['Peak Gust Direction'] >= 236.25) & (df['Peak Gust Direction'] < 258.75),'PGDirectionBkt' ]='WSW'
    df.loc[(df['Peak Gust Direction'] >= 258.75) & (df['Peak Gust Direction'] < 281.25),'PGDirectionBkt' ]='W'
    df.loc[(df['Peak Gust Direction'] >= 281.25) & (df['Peak Gust Direction'] < 303.75),'PGDirectionBkt' ]='WNW'
    df.loc[(df['Peak Gust Direction'] >= 303.75) & (df['Peak Gust Direction'] < 326.25),'PGDirectionBkt' ]='NW'
    df.loc[(df['Peak Gust Direction'] >= 326.25) & (df['Peak Gust Direction'] < 348.75),'PGDirectionBkt' ]='NNW'
    df.loc[(df['Peak Gust Direction'] >= 348.75) & (df['Peak Gust Direction'] < 360),'PGDirectionBkt' ]='N'

    # Deal with Peak Gust Direction
    df['PGDirectionBkt']=df['PGDirectionBkt'].fillna('NA')
    del df['Peak Gust Direction']
    
    

    finalDf = df.copy()
    pgdir_dct={}
    
    # Converting Objects to Floats
    #for col in finalDf.columns:
     #   finalDf[col] = finalDf[col].astype(float)
     
    continuous_columns=['Temp', 'Dew', 'T/D Spread', 'Wind Direction', 
                        'Wind Speed','Wind Gust', 'Altimeter', 'Vis', 'Layer 1', 'Peak Gust Speed', 'SchedDepart','SchedArriv']
    
    
    #categorical_columns=['Sky 1', 'WXCodes', 'PGDirectionBkt']
    
    print("******************************Input to Encoder******************************")
    print(finalDf.columns)
    
    filename = 'encoder_{}.sav'.format(airport)
    with (open(filename, "rb")) as openfile:
        biencoder = pickle.load(openfile)
    encoder = biencoder.fit(finalDf)
    #encoder = OneHotEncoder().fit(finalDf)
    finalDf_enc=encoder.transform(finalDf)
    print("******************************Output of Encoder******************************")
    print(finalDf_enc.columns)
    
    
    categorical_columns=[col for col in finalDf_enc.columns if col not in continuous_columns]
    
    df_corr = finalDf_enc[continuous_columns].corr(method='pearson', min_periods=1)
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > 0.8).any()
    #print(df_not_correlated.columns)
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    fnl_col_lst=list(un_corr_idx)
    fnl_col_lst+=categorical_columns
    finalDf_enc = finalDf_enc[fnl_col_lst]


    
    '''
    pgdl = finalDf['PGDirectionBkt'].drop_duplicates()
    for i,d in enumerate(pgdl):
        pgdir_dct[d]=i
    finalDf.replace({"PGDirectionBkt": pgdir_dct},inplace=True)
    WXDir={}
    wxl = finalDf['WXCodes'].drop_duplicates()
    for i,d in enumerate(wxl):
        WXDir[d]=i
    finalDf.replace({"WXCodes": WXDir},inplace=True)

    # Get dummies for Sky 1
    dummy = pd.get_dummies(finalDf['Sky 1'])
    finalDf = pd.concat([finalDf, dummy], axis = 1)
    del finalDf['Sky 1']
    '''

    
    X = finalDf_enc.copy()    
    print(X.columns)
    
    
    # Bucket response Variable into intervals of 10 minutes
    #bi_y = np.zeros((len(y),1))
    #intervals = np.arange(10, y.max(), 10)
    #for i in range(1, len(intervals)):
    #    bi_y[(y > intervals[i]) & (y <= intervals[i] + 10)] = i
    
    

    

    # OverSample due to Imbalance Data
    
    
    #y = preprocessing.label_binarize(y)
    #print(y)
    #y = LabelEncoder().fit_transform(y)
    # transform the dataset
    #oversample = SMOTE()
    #X_ros, y_ros = oversample.fit_resample(X, y)
    
    ros = RandomOverSampler(random_state= 42)

    # fit predictor and target variable
    X_ros, y_ros = ros.fit_resample(X, y)

    print('Original dataset shape', len(y))
    print('Resample dataset shape', len(y_ros))

    # Split the data into training and testing
    finalDf = pd.DataFrame(X_ros, columns = X.columns)
    
    
    train_X, test_X, train_y, test_y = train_test_split(X_ros, y_ros, test_size = 0.2, random_state = 42)
    
    return train_X, test_X, train_y, test_y

    
    

In [22]:
def test(flag):
    jsonFile = '/Users/jinhaenglee/Downloads/rf_BestModel_allAirPorts.json'
    with open(jsonFile) as f:
      data = json.load(f)
    
    models = list(data.keys())
    modelsDir = '/Users/jinhaenglee/Downloads/Models/'
    encodersDir = '/Users/jinhaenglee/Downloads/Models/Encoders/'
    inputDir = '/Users/jinhaenglee/Desktop/Georgia_Tech_OMSA/Practicum/Data/'
    outputDir = '/Users/jinhaenglee/Downloads/'
    files = glob.glob(inputDir + '*.xlsx')
    airports = []
    for f in files:
        airports.append(f.split('/')[-1].split(' ')[0])


    ap = []
    acc = []
    pre = []
    rec = []
    fs = []
    bal_ac = []
    for airport in ['ATL']:

        df = pd.read_excel(inputDir + airport + ' Final.xlsx', 'Data', header = 1)
        train_X, test_X, train_y, test_y = dataprep(df, airport)
        if flag == 1:
            test_X = test_X[test_y > 3]
            test_y = test_y[test_y > 3]

        for m in models:
            if airport in m:
                with (open(m, "rb")) as openfile:
                    RF = pickle.load(openfile)

                test_X_f = test_X[data[m]['features']]
                y_pred = RF.predict(test_X_f)
                accuracy,precision,recall,fscore,bal_accuracy = display_metrics(m, y_pred,test_y)
                ap.append(airport)
                acc.append(accuracy)
                pre.append(precision)
                rec.append(recall)
                fs.append(fscore)
                bal_ac.append(bal_accuracy)


    output = pd.DataFrame([])
    output['Airport'] = ap
    output['Accuracy'] = acc
    output['Recall'] = rec
    output['Precision'] = pre
    output['F1 score'] = fs
    output['Balanced Accuracy'] = bal_ac

    output.to_csv(outputDir + 'KnowDelay RandomForest Results.csv', index = False)




In [ ]:
def main():
    testFlag = 'Y'
    if testFlag == 'Y':
        test(1)
        
if __name__ == "__main__":
    main()

In [13]:
encoders

BinaryEncoder()